<a href="https://colab.research.google.com/github/srihasdama/4095_Final_Project/blob/main/Copy_of_RedditProject1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np
import pandas as pd

In [37]:
!pip install datasets
# FYI the accellerate -u is very important to avoid version issues with torch
!pip install transformers
!pip install accelerate -U

In [38]:
from datasets import load_dataset

In [39]:
dataset = load_dataset("fddemarco/pushshift-reddit-comments", streaming = True)

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [40]:
print(dataset)

IterableDatasetDict({
    train: IterableDataset({
        features: ['author', 'body', 'controversiality', 'created_utc', 'link_id', 'score', 'subreddit', 'subreddit_id', 'id'],
        n_shards: 50
    })
})


In [41]:
it = iter(dataset['train'])


In [42]:
extract_data = next(it)
#print(extract_data)

print(extract_data['body'])
#testing if we can get a sample comment

isn ' t this against the first amendment ? doesn ' t the first amendment give us the right to assemble and protest ?


In [43]:
data = {
    'column1_name': ['text'],
    'column2_name': ['score'],
    'column3_name': ['labels']
}

df = pd.DataFrame(data)

In [44]:
# Initialize an empty DataFrame with columns
df = pd.DataFrame(columns=['text', 'score', 'labels'])

nsfw_keywords = ['nsfw', 'adult', 'sex', 'porn', 'xxx', 'gonewild', 'nudes', 'slut', 'WTF',
'hentai', 'cum', 'rule34', 'pussy', 'onlyfans', 'milf', 'nude', 'boob']

mask = -df['labels'].str.contains('|'.join(nsfw_keywords))
df = df[mask]

for i in range(10000):  # only 10,000 rows
    extract_data = next(it)
    text = extract_data['body']
    score = extract_data['score']
    labels = extract_data['subreddit']

    new_row_data = {'text': text, "score": score, "labels": labels}
    df.loc[len(df)] = new_row_data

    #if i % 1000 == 0:
       # print(i)
        #print(new_row_data)





0
{'text': 'oil .', 'score': -4, 'labels': 'AskReddit'}
1000
{'text': 'is that a credible list ? how did they obtain their data ?', 'score': 1, 'labels': 'starcraft'}
2000
{'text': 'ok then : would you rather live in a street full of mps , or a street full of al - qaeda members , since this video seems to be implying that the threat level is the same .', 'score': -2, 'labels': 'unitedkingdom'}
3000
{'text': 'r / bad _ cop _ no _ latte coming soon .', 'score': 24, 'labels': 'Bad_Cop_No_Donut'}
4000
{'text': 'amen', 'score': 3, 'labels': 'treecomics'}
5000
{'text': "i remember that . . . it was a form message , wasn ' t it ? i used to end every one of my gm conversations , when they asked me ' is there anything else i can do to help you ' , with something along the lines of ' i know this is a shot in the dark , but could you possibly tell me an email address with which i might submit feedback and them promptly close the ticket , disallowing further communication with you ? ' i figured i 

In [45]:
df.head()

,text,score,labels
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [46]:
id2label = dict(enumerate(set(df['labels'])))

In [47]:
id2label

{0: 'EarthPorn',
 1: 'tf2trade',
 2: 'hiphop',
 3: 'mw3',
 4: 'Metal',
 5: 'Python',
 6: 'ferrets',
 7: 'ipad',
 8: 'bestof',
 9: 'firefly',
 10: 'Boxer',
 11: 'MinecraftCirclejerk',
 12: 'UFOs',
 13: 'picturesofiansleeping',
 14: 'obama',
 15: 'ForeverAloneDating',
 16: 'PHXMeetup',
 17: 'mylittlepony',
 18: 'hugeboobs',
 19: 'nba',
 20: 'bleach',
 21: 'unt',
 22: 'nostalgia',
 23: 'eagles',
 24: 'Hunting',
 25: 'Brooklyn',
 26: 'teenagers',
 27: 'CollegeBasketball',
 28: 'TrueReddit',
 29: 'Songwriters',
 30: 'secretsanta',
 31: 'mylittlehuman',
 32: 'Design',
 33: 'toycameras',
 34: 'ancientworldproblems',
 35: 'television',
 36: 'seduction',
 37: 'starcraft2_class',
 38: 'OFWGKTA',
 39: 'fibro',
 40: 'Millersville',
 41: 'TreesFortress2',
 42: 'jobs',
 43: 'nursing',
 44: 'ServerPorn',
 45: 'askseddit',
 46: 'Entrepreneur',
 47: 'xbox360',
 48: 'ipv6',
 49: 'MLPLounge',
 50: 'nintendo',
 51: 'askscience',
 52: 'arresteddevelopment',
 53: 'OttawaSenators',
 54: 'StAugustine',
 55: '

In [48]:
df.head()


,text,score,labels
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [49]:
from transformers import AutoTokenizer, RobertaForSequenceClassification
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load roberta model and tokenizer filling out the code below based on the documentation HINT: Click on on use in transformers and the code will be there
#model_name = "Roberta model"# code unnessisary but I like to use it to store the string name to make the code cleaner
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = len(id2label))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
nan_rows = df[df.isna().T.any().T]
nan_rows.head()

,text,score,labels


In [51]:
null_rows = df[df.isnull().any(axis=1)]
null_rows.head()

,text,score,labels


In [52]:
if np.isinf(df['score']).any():
  print("HELLO")

In [53]:
print(df.dtypes)

text      object
score      int64
labels    object
dtype: object


In [54]:
df.head()

,text,score,labels
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [55]:
from sklearn.model_selection import train_test_split

#TODO: Complete this line using the train_test_split function and set the test size to .21
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['labels'], test_size=0.21)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
# Reset the index
train_texts, train_labels = train_texts.reset_index(drop=True), train_labels.reset_index(drop=True)
val_texts, val_labels = val_texts.reset_index(drop=True), val_labels.reset_index(drop=True)

In [56]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, return_tensors = "pt")
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, return_tensors = "pt")

In [57]:
id2label

{0: 'EarthPorn',
 1: 'tf2trade',
 2: 'hiphop',
 3: 'mw3',
 4: 'Metal',
 5: 'Python',
 6: 'ferrets',
 7: 'ipad',
 8: 'bestof',
 9: 'firefly',
 10: 'Boxer',
 11: 'MinecraftCirclejerk',
 12: 'UFOs',
 13: 'picturesofiansleeping',
 14: 'obama',
 15: 'ForeverAloneDating',
 16: 'PHXMeetup',
 17: 'mylittlepony',
 18: 'hugeboobs',
 19: 'nba',
 20: 'bleach',
 21: 'unt',
 22: 'nostalgia',
 23: 'eagles',
 24: 'Hunting',
 25: 'Brooklyn',
 26: 'teenagers',
 27: 'CollegeBasketball',
 28: 'TrueReddit',
 29: 'Songwriters',
 30: 'secretsanta',
 31: 'mylittlehuman',
 32: 'Design',
 33: 'toycameras',
 34: 'ancientworldproblems',
 35: 'television',
 36: 'seduction',
 37: 'starcraft2_class',
 38: 'OFWGKTA',
 39: 'fibro',
 40: 'Millersville',
 41: 'TreesFortress2',
 42: 'jobs',
 43: 'nursing',
 44: 'ServerPorn',
 45: 'askseddit',
 46: 'Entrepreneur',
 47: 'xbox360',
 48: 'ipv6',
 49: 'MLPLounge',
 50: 'nintendo',
 51: 'askscience',
 52: 'arresteddevelopment',
 53: 'OttawaSenators',
 54: 'StAugustine',
 55: '

In [58]:
id2label_reversed = {v: k for k, v in id2label.items()}

In [59]:
id2label_reversed #word:number

{'EarthPorn': 0,
 'tf2trade': 1,
 'hiphop': 2,
 'mw3': 3,
 'Metal': 4,
 'Python': 5,
 'ferrets': 6,
 'ipad': 7,
 'bestof': 8,
 'firefly': 9,
 'Boxer': 10,
 'MinecraftCirclejerk': 11,
 'UFOs': 12,
 'picturesofiansleeping': 13,
 'obama': 14,
 'ForeverAloneDating': 15,
 'PHXMeetup': 16,
 'mylittlepony': 17,
 'hugeboobs': 18,
 'nba': 19,
 'bleach': 20,
 'unt': 21,
 'nostalgia': 22,
 'eagles': 23,
 'Hunting': 24,
 'Brooklyn': 25,
 'teenagers': 26,
 'CollegeBasketball': 27,
 'TrueReddit': 28,
 'Songwriters': 29,
 'secretsanta': 30,
 'mylittlehuman': 31,
 'Design': 32,
 'toycameras': 33,
 'ancientworldproblems': 34,
 'television': 35,
 'seduction': 36,
 'starcraft2_class': 37,
 'OFWGKTA': 38,
 'fibro': 39,
 'Millersville': 40,
 'TreesFortress2': 41,
 'jobs': 42,
 'nursing': 43,
 'ServerPorn': 44,
 'askseddit': 45,
 'Entrepreneur': 46,
 'xbox360': 47,
 'ipv6': 48,
 'MLPLounge': 49,
 'nintendo': 50,
 'askscience': 51,
 'arresteddevelopment': 52,
 'OttawaSenators': 53,
 'StAugustine': 54,
 'phot

In [60]:
import torch as pt

class SpamDataset(pt.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.values

    def __getitem__(self, idx):
        item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}
        if len(self.labels) > 0:  # Check if labels array has any elements
            labels = self.labels[idx]
            #print(labels)
            labels_converted = id2label_reversed[labels]
            #print(labels_converted)
            item["labels"] = pt.tensor(labels_converted)
        return item
    def __len__(self):
        print(len(self.labels))
        return len(self.labels)

def list_of_dicts_to_dict_of_lists(d):
  dic = d[0]
  keys = dic.keys()
  values = [dic.values() for dic in d]
  return {k: list(v) for k,v in zip(keys, zip(*values))}

In [61]:
Train_Dataset = SpamDataset(train_encodings, train_labels)


In [62]:
Val_Dataset = SpamDataset(val_encodings, val_labels)

In [63]:
!pip install datasets transformers torch evaluate nltk rouge_score

In [64]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate


In [65]:
from sklearn.metrics import accuracy_score

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [66]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [67]:
%env CUDA_LAUNCH_BLOCKING

'1'

In [68]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs, let's try 1 for now
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = Train_Dataset,
    eval_dataset = Val_Dataset,
    compute_metrics = compute_metrics
    #TODO: figure out what 5 params we need here. There should be 5 and the last is how we compute the metrics later . . .
)

7900


In [69]:
#set(id2label_reversed.values)

In [70]:
# TODO: Train the model (HINT: this is 1 short line of code just calling the trainer)
trainer.train()

7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
AskReddit
265
AskReddit
265
gaming
396
politics
177
AskReddit
265
funny
145
techsupport
664
skyrim
106
AskReddit
265
fffffffuuuuuuuuuuuu
854
guns
453
applehelp
736
tea
93
funny
145
TheRabbitHole
547
pics
266


<ipython-input-60-df3c2ef05379>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,6.810200
20,6.787000
30,6.799700
40,6.753500
50,6.765500
60,6.776400
70,6.728000
80,6.729100
90,6.539900
100,6.426000


Streaming output truncated to the last 5000 lines.
LosAngeles
122
mylittlepony
17
SteamTrade
715
saudiarabia
646
AskReddit
265
WTF
836
AusCraft
202
penguins
613
haskell
594
leagueoflegends
231
Homebrewing
824
gameswap
537
RenewableEnergy
326
politics
177
Libertarian
103
pics
266
fffffffuuuuuuuuuuuu
854
gaming
396
worldnews
92
trees
63
atheism
115
AskReddit
265
AdviceAnimals
804
AskReddit
265
AskReddit
265
atheism
115
fffffffuuuuuuuuuuuu
854
AskReddit
265
tf2
823
funny
145
politics
177
Games
666
LifeProTips
351
skyrim
106
Borderlands
311
SteamGameSwap
315
politics
177
Android
355
Bad_Cop_No_Donut
191
movies
249
self
362
LucidDreaming
749
IAmA
76
pics
266
ShitRedditSays
58
community
608
malefashionadvice
639
Cooking
421
comics
142
pics
266
PHXMeetup
16
IAmA
76
Christianity
428
law
588
pics
266
hiphop
2
Marvel
329
Games
666
worldnews
92
AskReddit
265
nba
19
fffffffuuuuuuuuuuuu
854
Cooking
421
funny
145
AskReddit
265
gaming
396
firstworldproblems
440
AskReddit
265
MensRights
740
fffffffuuu

<ipython-input-60-df3c2ef05379>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


Streaming output truncated to the last 5000 lines.
3
IAmA
76
funny
145
gaming
396
MensRights
740
AskReddit
265
palmtrees
669
swtor
645
AskReddit
265
leagueoflegends
231
AskReddit
265
fffffffuuuuuuuuuuuu
854
google
742
videos
580
politics
177
politics
177
sanfrancisco
733
AdviceAnimals
804
gaming
396
newjersey
346
fffffffuuuuuuuuuuuu
854
Python
5
AskReddit
265
fffffffuuuuuuuuuuuu
854
palmtrees
669
askscience
51
IAmA
76
AskReddit
265
Drugs
157
AskReddit
265
mw3
3
movies
249
fffffffuuuuuuuuuuuu
854
pics
266
doctorwho
767
AskReddit
265
gaymers
881
atheism
115
politics
177
AskReddit
265
leagueoflegends
231
movies
249
AskReddit
265
gaming
396
freebies
809
nfl
447
todayilearned
548
atheism
115
lgbt
849
fantasybball
838
Games
666
IAmA
76
skyrim
106
funny
145
AskReddit
265
politics
177
politics
177
VirginiaTech
528
gaming
396
nba
19
torrents
882
MMFB
505
Libertarian
103
starcraft
314
politics
177
Minecraft
146
starcraft
314
AskReddit
265
mylittlepony
17
AskReddit
265
skyrim
106
AskReddit
265
fu

TrainOutput(global_step=988, training_loss=5.104928553345715, metrics={'train_runtime': 298.5045, 'train_samples_per_second': 26.465, 'train_steps_per_second': 3.31, 'total_flos': 2095093833216000.0, 'train_loss': 5.104928553345715, 'epoch': 1.0})

In [71]:
#TODO use the evaluate method on the trainer to get and print the results. Feel free to look at Huggingface Docs
results = trainer.evaluate()

2100
2100
2100
2100
swtor
645
Minecraft
146
atheism
115
AskReddit
265
atheism
115
hockey
534
eu3
212
explainlikeimfive
835
gaming
396
technology
794
funny
145
atheism
115
Music
771
AskReddit
265
Fitness
562
AskReddit
265
startrek
158
soccer
419
unitedkingdom
407
Drugs
157
WTF
836
nba
19
funny
145
funny
145
SteamTrade
715
PS3
368
atheism
115
ShitRedditSays
58
fffffffuuuuuuuuuuuu
854
AskReddit
265
pics
266
VirginiaTech
528
soccer
419
AskReddit
265
WTF
836
AskReddit
265
Cooking
421
Frugal
164
funny
145
ottawa
228
occupywallstreet
270
learnprogramming
169
atheism
115
SteamTrade
715
AskReddit
265
ps3bf3
449
fffffffuuuuuuuuuuuu
854
aww
772
MLPLounge
49
AdviceAnimals
804
AskReddit
265
unitedkingdom
407
atheism
115
movies
249
pics
266
Anarchism
519
AskReddit
265
SteamGameSwap
315
politics
177
leagueoflegends
231
MensRights
740
harrypotter
181
AskReddit
265
IAmA
76


<ipython-input-60-df3c2ef05379>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


2100
2100


socialism
309
atheism
115
gaming
396
r4r
448
Fitness
562
GirlGamers
384
movies
249
TrueReddit
28
atheism
115
mylittlepony
17
fffffffuuuuuuuuuuuu
854
gonewild
88
AskReddit
265
motorcycles
654
Music
771
AskReddit
265
ar15
672
Drugs
157
Libertarian
103
SteamGameSwap
315
gameswap
537
anime
601
Android
355
GirlGamers
384
Cooking
421
ShitRedditSays
58
pics
266
dahouse
684
askscience
51
SteamGameSwap
315
SteamGameSwap
315
leagueoflegends
231
2100
Demotivational
150
AskReddit
265
technology
794
Cooking
421
pics
266
SteamGameSwap
315
SteamTrade
715
canada
718
food
880
politics
177
aww
772
trees
63
pics
266
poker
750
WTF
836
AskReddit
265
aww
772
trees
63
wallpapers
104
pics
266
Eve
211
AskReddit
265
pics
266
leagueoflegends
231
Minecraft
146
AskReddit
265
ShitRedditSays
58
AdviceAnimals
804
pics
266
MLPLounge
49
SteamGameSwap
315
masseffect
450
2100
gaming
396
AndroidQuestions
414
wow
622
politics
177
pics
266
firstimpression
168
worldnews
92
AskReddit
265
washu
334
AdviceAnimals
804
WTF
836
te

In [72]:
# TODO: Save the model
trainer.save_model('my_model')

In [73]:
# TODO: Now to finish off I want you to load the model you trained and saved and write a fake spam email for it mthen have the model classify it
model = AutoModelForSequenceClassification.from_pretrained("my_model")
msg = "I want to download minecraft mods"
msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")

outputs = model(**msg_encodings)
print(f"outputs: {outputs}")




outputs: SequenceClassifierOutput(loss=None, logits=tensor([[-9.1665e-01,  1.2231e+00, -1.0408e+00,  8.2711e-01, -5.2322e-02,
         -2.5537e+00, -3.0702e+00, -3.0727e+00, -8.6466e-01, -1.2022e+00,
         -2.9752e+00, -3.3379e+00, -1.4947e+00, -1.8293e+00, -1.6048e+00,
         -3.0943e+00, -1.3374e+00,  1.2598e+00, -2.1223e+00,  5.0501e-01,
         -1.7298e+00, -1.3747e+00, -1.8077e+00, -2.8486e+00, -1.5050e+00,
         -2.1940e+00, -2.4297e-01, -1.4459e+00, -6.7593e-01, -2.2325e+00,
         -5.6673e-01, -7.8090e-01, -2.9485e+00, -1.9802e+00, -1.6793e+00,
         -9.2514e-01, -9.2100e-01, -1.5337e+00, -2.9456e+00, -2.4506e+00,
         -2.8963e+00, -1.0968e+00, -1.6309e+00, -3.0370e+00, -1.8543e+00,
         -1.4580e+00, -1.8932e+00, -7.6048e-01, -2.7107e+00,  3.1891e-01,
         -7.2139e-01,  3.7263e-01, -2.8400e+00, -3.0278e+00, -2.6049e+00,
         -3.1220e-01, -9.4813e-01, -8.7596e-01, -2.0797e-01, -1.7605e+00,
         -1.6338e+00, -1.3649e+00, -1.3995e+00,  1.7671e+00,

In [74]:
outputs.logits.shape

torch.Size([1, 887])

In [75]:
len(id2label)

887

In [76]:
probabilities = torch.softmax(outputs.logits[0], dim = 0 )
print(probabilities)

tensor([7.7923e-04, 6.6213e-03, 6.8828e-04, 4.4563e-03, 1.8494e-03, 1.5160e-04,
        9.0450e-05, 9.0221e-05, 8.2082e-04, 5.8567e-04, 9.9463e-05, 6.9202e-05,
        4.3713e-04, 3.1282e-04, 3.9159e-04, 8.8294e-05, 5.1159e-04, 6.8690e-03,
        2.3337e-04, 3.2291e-03, 3.4556e-04, 4.9286e-04, 3.1966e-04, 1.1289e-04,
        4.3267e-04, 2.1724e-04, 1.5284e-03, 4.5900e-04, 9.9130e-04, 2.0903e-04,
        1.1057e-03, 8.9252e-04, 1.0215e-04, 2.6902e-04, 3.6345e-04, 7.7264e-04,
        7.7584e-04, 4.2044e-04, 1.0245e-04, 1.6807e-04, 1.0762e-04, 6.5075e-04,
        3.8149e-04, 9.3497e-05, 3.0512e-04, 4.5347e-04, 2.9346e-04, 9.1094e-04,
        1.2957e-04, 2.6808e-03, 9.4725e-04, 2.8287e-03, 1.1386e-04, 9.4368e-05,
        1.4403e-04, 1.4262e-03, 7.5508e-04, 8.1159e-04, 1.5829e-03, 3.3510e-04,
        3.8037e-04, 4.9774e-04, 4.8078e-04, 1.1408e-02, 2.6540e-04, 9.4784e-05,
        6.1316e-04, 2.1957e-04, 1.0113e-04, 9.0885e-04, 5.5370e-04, 1.1369e-04,
        7.5902e-04, 1.0080e-03, 3.1079e-

In [77]:
sort_prob = probabilities.sort(descending = True)

In [78]:
print(sort_prob)

torch.return_types.sort(
values=tensor([1.5406e-01, 7.4396e-02, 6.6965e-02, 2.6927e-02, 2.3994e-02, 1.8896e-02,
        1.5541e-02, 1.2810e-02, 1.2156e-02, 1.1791e-02, 1.1500e-02, 1.1408e-02,
        9.0727e-03, 8.9501e-03, 8.9269e-03, 8.7011e-03, 7.7218e-03, 7.5380e-03,
        6.8690e-03, 6.8679e-03, 6.6213e-03, 6.0896e-03, 5.9135e-03, 5.7953e-03,
        5.7018e-03, 5.5347e-03, 5.4782e-03, 5.4687e-03, 5.1710e-03, 4.7398e-03,
        4.6971e-03, 4.4563e-03, 4.2046e-03, 4.1183e-03, 3.8139e-03, 3.6100e-03,
        3.4113e-03, 3.2987e-03, 3.2291e-03, 3.1596e-03, 2.8694e-03, 2.8287e-03,
        2.8209e-03, 2.8078e-03, 2.7385e-03, 2.6808e-03, 2.5664e-03, 2.5466e-03,
        2.5033e-03, 2.4828e-03, 2.4643e-03, 2.4238e-03, 2.4093e-03, 2.3660e-03,
        2.3434e-03, 2.3127e-03, 2.2346e-03, 2.2329e-03, 2.1129e-03, 2.1022e-03,
        2.0937e-03, 2.0798e-03, 2.0482e-03, 2.0097e-03, 1.9526e-03, 1.8924e-03,
        1.8861e-03, 1.8742e-03, 1.8494e-03, 1.7938e-03, 1.7591e-03, 1.7565e-03,
        

In [79]:
def print_out_top_x_probs(x,phrase):
  msg = phrase
  msg = "I want to download minecraft mods"
  print('Here are the top 10 most subreddits with similar')
  msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")
  probabilities = torch.softmax(outputs.logits[0], dim = 0 )
  sort_prob = probabilities.sort(descending = True)

  outputs = model(**msg_encodings)
  for n in range(x):
    sort_probs = probs
    tensor_probs = sort_probs.values[n]
    index = sort_probs.indices[n]
    sort_prob_index_int = index.item()
    subreddit_string = id2label[sort_prob_index_int]
    print(f'{n+1}. {subreddit_string} ({tensor_probs * 100}%)')

In [90]:
sort_prob.indices[0].item()

396

In [94]:
def print_out_top_x_probs(phrase):
  phrase_encoding = msg_encodings = tokenizer(phrase, truncation=True, padding=True, return_tensors = "pt")
  output = model(**phrase_encoding)
  probabilities = torch.softmax(output.logits[0], dim = 0 )
  sort_prob = probabilities.sort(descending = True)

  for n in range(10):
    tensor_probs = sort_prob.values[n]
    index = sort_prob.indices[n]
    sort_prob_index_int = index.item()
    subreddit_string = id2label[sort_prob_index_int]
    print(f'{n+1}. {subreddit_string} ({tensor_probs * 100}%)')

In [95]:
phrase = "green paint"
print_out_top_x_probs(phrase)

1. AskReddit (20.27960205078125%)
2. funny (10.783051490783691%)
3. pics (9.65079116821289%)
4. fffffffuuuuuuuuuuuu (3.9802730083465576%)
5. WTF (3.0142176151275635%)
6. politics (2.850417137145996%)
7. trees (2.6972923278808594%)
8. gaming (2.3545448780059814%)
9. AdviceAnimals (1.9897096157073975%)
10. IAmA (1.5288171768188477%)


In [ ]:
# it's time to graph!

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy
    }


In [ ]:
import matplotlib.pyplot as plt

# this is fake data, replace with real data
epochs = 1
loss = [0.30, 0.24, 0.20]
f1_scores = [0.82, 0.85, 0.87]

plt.figure(figsize=(12, 5))

# Plotting loss
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='Training Loss', marker='o')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Plotting F1 Score
plt.subplot(1, 2, 2)
plt.plot(epochs, f1_scores, label='F1 Score', marker='o', color='r')
plt.title('F1 Score per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()
